In [ ]:
%pip install fake_useragent
import  csv, fake_useragent, os, requests, time
from bs4 import BeautifulSoup

url ='https://www.the-models.de'
url2 = [f'{url}/damen/seite/{num}' for num in range(2,16)]
url2.insert(0,url)
basedir = "the-models.de"

: 

In [36]:
ua = fake_useragent.UserAgent()
print(url2)
error_dict=dict()

Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727)
['https://www.the-models.de', 'https://www.the-models.de/damen/seite/2', 'https://www.the-models.de/damen/seite/3', 'https://www.the-models.de/damen/seite/4', 'https://www.the-models.de/damen/seite/5', 'https://www.the-models.de/damen/seite/6', 'https://www.the-models.de/damen/seite/7', 'https://www.the-models.de/damen/seite/8', 'https://www.the-models.de/damen/seite/9', 'https://www.the-models.de/damen/seite/10', 'https://www.the-models.de/damen/seite/11', 'https://www.the-models.de/damen/seite/12', 'https://www.the-models.de/damen/seite/13', 'https://www.the-models.de/damen/seite/14', 'https://www.the-models.de/damen/seite/15']


In [42]:

# FUNCS
def download_pics(pics:list, k)-> None:
        for link in pics:
            path = f"{basedir}/{k}/{link.split('/')[-1]}"
            if not os.path.exists(path):
                with open(path, 'wb') as f:
                        f.write(requests.get(link).content)

def parse(lst : list,) -> dict:
    model_info = dict()
    translate = ["HEIGHT", "DRESS", "BUST", "CUP SIZE","WAIST","SHOE","HIP","HAIR"]
    for i,val in enumerate(lst[:-1]):
        val=val.split()
        model_info[translate[i]] = "".join(val[1:])
    return model_info

models_details = list()

In [44]:
        
def scrape(link):
  try:
    soup = BeautifulSoup(requests.get(link, headers = {'User-Agent': ua.random}).content, 'html5lib')
    models = soup.findAll('div', class_='box-bild-20-prozent')
    models_name = [name.text.strip() for name in models]
    models_link = [link.a['href'] for link in models]
    models_dict = dict(zip(models_name,models_link))
    models_dict


    for k,v in models_dict.items():
      print(k)
      os.makedirs(f"{basedir}/{k}", exist_ok=True)
      soup1 = BeautifulSoup(requests.get(f"{url}{v}",headers = {'User-Agent': ua.random}).content, 'html5lib')
      pics = [pic['href'] for pic in soup1.find('div', class_='grid').findAll('a')]
      raw_features = [feature for feat in soup1.find('div', class_='sedcard-details-box-1').findAll('div') for feature in feat.text.strip().split('\n') ]
      feat_dict = parse(raw_features)
      feat_dict['NAME'] = k
      feat_dict['URL'] = v
      download_pics(pics, k)


        
      models_details.append(feat_dict)
  except Exception as e:
    error_dict[link]= str(e)
  print(f"Link: {link}\tModels {len(models_details)}")

In [ ]:
# THE MAIN CELL
for link in url2:
    scrape(link)
    time.sleep(5)

"HEIGHT", "DRESS", "BUST", "CUP SIZE","WAIST","SHOE","HIP","HAIR"


In [47]:
def list_to_csv(lst):
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL","HEIGHT", "DRESS", "BUST", "CUP SIZE","WAIST","SHOE","HIP","HAIR"])
        c.writeheader()
        for row in lst:
            c.writerow(row)
list_to_csv(models_details)

In [1]:
print(error_dict)

NameError: name 'error_dict' is not defined